In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/dataset-for-collaborative-filters/songsDataset.csv


Installing PySpark

In [2]:
!pip install pyspark

     |████████████████████████████████| 217.8 MB 4.5 kB/s 
     |████████████████████████████████| 197 kB 49.1 MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218258791 sha256=636337936963d3e37c553a275c32a5f79a9a950731168c146ac89fef0ed72259
  Stored in directory: /root/.cache/pip/wheels/84/30/e3/c51c5cd0229631e662d29d7b578a3e5949a4c8db033ffb70aa
Successfully built pyspark


In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

Starting Spark Session with Context

In [4]:
sc = SparkContext('local')
spark = SparkSession(sc)

To read the data, we first need to desin an scheme with appropriate data types, matching with CSV file inputs. Our dataset has three fields namely userID, songID, and rating.

In [5]:
input_schema = StructType([
    StructField('userID',IntegerType(), False),
    StructField('songID',IntegerType(), False),
    StructField('rating',IntegerType(), False),
])
data = spark.read.csv(
    '../input/dataset-for-collaborative-filters/songsDataset.csv', header=True, schema=input_schema
).cache()

Now, we split the data into training and test set, 78% testing and 22% test.

In [6]:
(training_data, test_data) = data.randomSplit([0.78, 0.22])

Now that we have required training and test dataset, we build collaborative filtering Alternate Least Squares model offered by PySpark. To solve the cold square problem, we use 'drop' strategy. We run our model for iterations.

In [7]:
als = ALS(maxIter=10, regParam=0.01, userCol="userID", itemCol="songID", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

NameError: name 'training' is not defined

Now we make predictions.

In [8]:
predictions = model.transform(test)

NameError: name 'model' is not defined

We evaulate our model using MSE i.e., Mean Square Error metric.

In [9]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")

In [10]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

NameError: name 'predictions' is not defined

We find that the RMSE is 5.97. We can increase the number of iterations to better our result.